#__Colab Stay Alive__

>Code 1</br>(credit to rockyourcode)
<pre><code>function ClickConnect() {
  console.log('Working')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}

setInterval(ClickConnect, 60000)</code></pre>

</br>

> Code 2</br>(credit to Kavyajeet Bora on stack overflow)
<pre><code>function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)</code></pre>

</br>

> Code 3
<pre><code>function ClickConnect(){
  console.log("Connnect Clicked - Start"); 
  document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
  console.log("Connnect Clicked - End"); 
};
setInterval(ClickConnect, 60000)</code></pre>

</br>

> Code 4</br>(credit to Stephane Belemkoabga on stack overflow)
<pre><code>function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-connect-button").click() 
}
setInterval(ClickConnect,60000)</code></pre>

</br>

> Code 5</br>(credit to unknown)
<pre><code>function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,20000);</code></pre>

</br>

> Code 6</br>(credit to unknown)
<pre><code>function KeepClicking(){
   console.log("Clicking");
   document.querySelector("colab-toolbar-button#connect").click()
}setInterval(KeepClicking,60000)</code></pre>

</br>

#__Mount with Google Drive File Stream__

In [ ]:
#@markdown <h3>Mount GDrive to /content/drive</h3><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

#<img src="https://raw.githubusercontent.com/rclone/rclone/master/graphics/logo/svg/logo_symbol_color.svg" height="45" align="left"/><b><font size=6>Rclone</font></b>

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=5>← Install Rclone</font>
build_version = "stable" #@param ["stable", "beta"]
 

automatically_clear_cell_output = True  # @param{type: "boolean"}
# ================================================================ #
 
import os
import IPython
from IPython.display import clear_output
 
 
if build_version == "stable":
    !curl https://rclone.org/install.sh | sudo bash
else:
    !curl https://rclone.org/install.sh | sudo bash -s beta
 
 
try:
    os.makedirs("/root/.config/rclone", exist_ok=True)
except OSError as error:
    pass
 
 
if automatically_clear_cell_output is True:
    clear_output()
else:
    pass

### <font size=6>Rclone Configuration</font>

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] rclone CLI Configuration</font>
# @markdown <p>Run this cell to create and/or edit an rclone configuration.</p>
# @markdown > After you have created a configuration, download the configuration file.</br>In the next time you want to mount an rclone drive, simply import the configuration file.
USE_FREE_TOKEN = True
TOKEN = ""  #
REGION = "US"
Tunneling = "argo_tunnel_(cloudflare)" #@param ["argo_tunnel_(cloudflare)", "localhost.run", "ngrok"]
 
# @markdown ---
automatically_clear_cell_output = True  # @param{type: "boolean"}
 
if Tunneling == "argo_tunnel_(cloudflare)":
    PORT_FORWARD = "argotunnel"
elif Tunneling == "localhost.run":
    PORT_FORWARD = "localhost"
elif Tunneling == "ngrok":
    PORT_FORWARD = "ngrok"
# ================================================================ #
 
import os, urllib.request, IPython
from IPython.display import HTML, clear_output
 
HOME = os.path.expanduser("~")
runW = get_ipython()
 
if not os.path.exists("/root/.ipython/mixlab.py"):
  from shlex import split as _spl
  from subprocess import run
 
  shellCmd = "wget -qq https://shirooo39.github.io/MiXLab/resources/mixlab.py \
                  -O /root/.ipython/mixlab.py"
  run(_spl(shellCmd))
 
from mixlab import (
    prepareSession,
    rcloneConfigurationPath,
    runSh,
    PortForward_wrapper
)
 
import codecs, contextlib, locale, os, pty, select, signal, subprocess, sys, termios, time
from IPython.utils import text
import six
from google.colab import _ipython
from google.colab import _message
from google.colab.output import _tags
 
# Linux read(2) limits to 0x7ffff000 so stay under that for clarity.
_PTY_READ_MAX_BYTES_FOR_TEST = 2**20  # 1MB
 
_ENCODING = 'UTF-8'
 
class ShellResult(object):
  """Result of an invocation of the shell magic.
 
  Note: This is intended to mimic subprocess.CompletedProcess, but has slightly
  different characteristics, including:
    * CompletedProcess has separate stdout/stderr properties. A ShellResult
      has a single property containing the merged stdout/stderr stream,
      providing compatibility with the existing "!" shell magic (which this is
      intended to provide an alternative to).
    * A custom __repr__ method that returns output. When the magic is invoked as
      the only statement in the cell, Python prints the string representation by
      default. The existing "!" shell magic also returns output.
  """
 
  def __init__(self, args, returncode, command_output):
    self.args = args
    self.returncode = returncode
    self.output = command_output
 
  def check_returncode(self):
    if self.returncode:
      raise subprocess.CalledProcessError(
          returncode=self.returncode, cmd=self.args, output=self.output)
 
  def _repr_pretty_(self, p, cycle):  # pylint:disable=unused-argument
    # Note: When invoking the magic and not assigning the result
    # (e.g. %shell echo "foo"), Python's default semantics will be used and
    # print the string representation of the object. By default, this will
    # display the __repr__ of ShellResult. Suppress this representation since
    # the output of the command has already been displayed to the output window.
    if cycle:
      raise NotImplementedError
 
 
def _configure_term_settings(pty_fd):
  term_settings = termios.tcgetattr(pty_fd)
  # ONLCR transforms NL to CR-NL, which is undesirable. Ensure this is disabled.
  # http://man7.org/linux/man-pages/man3/termios.3.html
  term_settings[1] &= ~termios.ONLCR
 
  # ECHOCTL echoes control characters, which is undesirable.
  term_settings[3] &= ~termios.ECHOCTL
 
  termios.tcsetattr(pty_fd, termios.TCSANOW, term_settings)
 
 
def _run_command(cmd, clear_streamed_output):
  """Calls the shell command, forwarding input received on the stdin_socket."""
  locale_encoding = locale.getpreferredencoding()
  if locale_encoding != _ENCODING:
    raise NotImplementedError(
        'A UTF-8 locale is required. Got {}'.format(locale_encoding))
 
  parent_pty, child_pty = pty.openpty()
  _configure_term_settings(child_pty)
 
  epoll = select.epoll()
  epoll.register(
      parent_pty,
      (select.EPOLLIN | select.EPOLLOUT | select.EPOLLHUP | select.EPOLLERR))
 
  try:
    temporary_clearer = _tags.temporary if clear_streamed_output else _no_op
 
    with temporary_clearer(), _display_stdin_widget(
        delay_millis=500) as update_stdin_widget:
      # TODO(b/115531839): Ensure that subprocesses are terminated upon
      # interrupt.
      p = subprocess.Popen(
          cmd,
          shell=True,
          executable='/bin/bash',
          stdout=child_pty,
          stdin=child_pty,
          stderr=child_pty,
          close_fds=True)
      # The child PTY is only needed by the spawned process.
      os.close(child_pty)
 
      return _monitor_process(parent_pty, epoll, p, cmd, update_stdin_widget)
  finally:
    epoll.close()
    os.close(parent_pty)
 
 
class _MonitorProcessState(object):
 
  def __init__(self):
    self.process_output = six.StringIO()
    self.is_pty_still_connected = True
 
 
def _monitor_process(parent_pty, epoll, p, cmd, update_stdin_widget):
  """Monitors the given subprocess until it terminates."""
  state = _MonitorProcessState()
 
  # A single UTF-8 character can span multiple bytes. os.read returns bytes and
  # could return a partial byte sequence for a UTF-8 character. Using an
  # incremental decoder is incrementally fed input bytes and emits UTF-8
  # characters.
  decoder = codecs.getincrementaldecoder(_ENCODING)()
 
  num_interrupts = 0
  echo_status = None
  while True:
    try:
      result = _poll_process(parent_pty, epoll, p, cmd, decoder, state)
      if result is not None:
        return result
      term_settings = termios.tcgetattr(parent_pty)
      new_echo_status = bool(term_settings[3] & termios.ECHO)
      if echo_status != new_echo_status:
        update_stdin_widget(new_echo_status)
        echo_status = new_echo_status
    except KeyboardInterrupt:
      try:
        num_interrupts += 1
        if num_interrupts == 1:
          p.send_signal(signal.SIGINT)
        elif num_interrupts == 2:
          # Process isn't responding to SIGINT and user requested another
          # interrupt. Attempt to send SIGTERM followed by a SIGKILL if the
          # process doesn't respond.
          p.send_signal(signal.SIGTERM)
          time.sleep(0.5)
          if p.poll() is None:
            p.send_signal(signal.SIGKILL)
      except KeyboardInterrupt:
        # Any interrupts that occur during shutdown should not propagate.
        pass
 
      if num_interrupts > 2:
        # In practice, this shouldn't be possible since
        # SIGKILL is quite effective.
        raise
 
 
def _poll_process(parent_pty, epoll, p, cmd, decoder, state):
  """Polls the process and captures / forwards input and output."""
 
  terminated = p.poll() is not None
  if terminated:
    termios.tcdrain(parent_pty)
    # We're no longer interested in write events and only want to consume any
    # remaining output from the terminated process. Continuing to watch write
    # events may cause early termination of the loop if no output was
    # available but the pty was ready for writing.
    epoll.modify(parent_pty,
                 (select.EPOLLIN | select.EPOLLHUP | select.EPOLLERR))
 
  output_available = False
 
  events = epoll.poll()
  input_events = []
  for _, event in events:
    if event & select.EPOLLIN:
      output_available = True
      raw_contents = os.read(parent_pty, _PTY_READ_MAX_BYTES_FOR_TEST)
      import re
      decoded_contents = re.sub(r"http:\/\/127.0.0.1:53682", Server["url"], 
                                decoder.decode(raw_contents))
      sys.stdout.write(decoded_contents)
      state.process_output.write(decoded_contents)
 
    if event & select.EPOLLOUT:
      # Queue polling for inputs behind processing output events.
      input_events.append(event)
 
    # PTY was disconnected or encountered a connection error. In either case,
    # no new output should be made available.
    if (event & select.EPOLLHUP) or (event & select.EPOLLERR):
      state.is_pty_still_connected = False
 
  for event in input_events:
    # Check to see if there is any input on the stdin socket.
    # pylint: disable=protected-access
    input_line = _message._read_stdin_message()
    # pylint: enable=protected-access
    if input_line is not None:
      # If a very large input or sequence of inputs is available, it's
      # possible that the PTY buffer could be filled and this write call
      # would block. To work around this, non-blocking writes and keeping
      # a list of to-be-written inputs could be used. Empirically, the
      # buffer limit is ~12K, which shouldn't be a problem in most
      # scenarios. As such, optimizing for simplicity.
      input_bytes = bytes(input_line.encode(_ENCODING))
      os.write(parent_pty, input_bytes)
 
  # Once the process is terminated, there still may be output to be read from
  # the PTY. Wait until the PTY has been disconnected and no more data is
  # available for read. Simply waiting for disconnect may be insufficient if
  # there is more data made available on the PTY than we consume in a single
  # read call.
  if terminated and not state.is_pty_still_connected and not output_available:
    sys.stdout.flush()
    command_output = state.process_output.getvalue()
    return ShellResult(cmd, p.returncode, command_output)
 
  if not output_available:
    # The PTY is almost continuously available for reading input to provide
    # to the underlying subprocess. This means that the polling loop could
    # effectively become a tight loop and use a large amount of CPU. Add a
    # slight delay to give resources back to the system while monitoring the
    # process.
    # Skip this delay if we read output in the previous loop so that a partial
    # read doesn't unnecessarily sleep before reading more output.
    # TODO(b/115527726): Rather than sleep, poll for incoming messages from
    # the frontend in the same poll as for the output.
    time.sleep(0.1)
 
 
@contextlib.contextmanager
def _display_stdin_widget(delay_millis=0):
  """Context manager that displays a stdin UI widget and hides it upon exit.
 
  Args:
    delay_millis: Duration (in milliseconds) to delay showing the widget within
      the UI.
 
  Yields:
    A callback that can be invoked with a single argument indicating whether
    echo is enabled.
  """
  shell = _ipython.get_ipython()
  display_args = ['cell_display_stdin', {'delayMillis': delay_millis}]
  _message.blocking_request(*display_args, parent=shell.parent_header)
 
  def echo_updater(new_echo_status):
    # Note: Updating the echo status uses colab_request / colab_reply on the
    # stdin socket. Input provided by the user also sends messages on this
    # socket. If user input is provided while the blocking_request call is still
    # waiting for a colab_reply, the input will be dropped per
    # https://github.com/googlecolab/colabtools/blob/56e4dbec7c4fa09fad51b60feb5c786c69d688c6/google/colab/_message.py#L100.
    update_args = ['cell_update_stdin', {'echo': new_echo_status}]
    _message.blocking_request(*update_args, parent=shell.parent_header)
 
  yield echo_updater
 
  hide_args = ['cell_remove_stdin', {}]
  _message.blocking_request(*hide_args, parent=shell.parent_header)
 
 
@contextlib.contextmanager
def _no_op():
  yield
 
prepareSession()
 
Server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['rcloneConfiguration', 53682, 'http']], 'REGION.lower', [f"{HOME}/.ngrok2/rcloneConfiguration.yml", 4074]).start('rcloneConfiguration', displayB=False, v=False)
 
printData = """
Before finishing the configuration, you will be redirected to an address.
Replace the address http://127.0.0.0:53682 with {}""".format(Server['url'])
print(printData)
display(HTML('</br>(Click <a href="https://raw.githubusercontent.com/tofuliang/Google-Colab-CloudTorrent/master/src/rclone_config_create.gif" target="_blank">here</a> to see how to do it)</br></br>'))
print(f"{Server['url']}", end="\n\n")
_run_command(f"rclone config --config {rcloneConfigurationPath}/rclone.conf", False)
 
 
if automatically_clear_cell_output is True:
    clear_output()
else:
    pass

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Download Configuration File </font>
# @markdown <p>Download configuration file from the VM into your local machine.</p>
 
# @markdown ---
MODE = "RCONFIG" # @param ['UTILS', 'RCONFIG']
# ================================================================ #
 
from IPython.display import clear_output
from google.colab import files
 
def downloadFile():
  if MODE == "UTILS":
      filePath = "/root/.ipython/mixlab.py"
  elif MODE == "RCONFIG":
      filePath = f"{rcloneConfigurationPath}/rclone.conf"
  else:
      pass
  try:
    files.download(filePath)
  except FileNotFoundError:
    print("File not found!")
 
if __name__ == "__main__":
  downloadFile()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Upload Configuration File </font>
# @markdown <p>If you already have an rclone configuration file, you can upload it by running this cell.</p>
 
# @markdown ---
MODE = "RCONFIG" # @param ['UTILS', 'RCONFIG', 'RCONFIG_append', "GENERATELIST"]
REMOTE = "mnc" # @param {type:"string"}
QUERY_PATTERN = "" # @param {type:"string"}
# @markdown > <p><font size=2px>For those who are unable to upload local file: <a href="https://stackoverflow.com/a/58661947">StackOverflow</a></font></p>
# ================================================================ #
 
from os import path as _p
from IPython.display import HTML, clear_output
 
if not _p.exists("/root/.ipython/mixlab.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec
 
    shellCmd = "wget -qq https://shirooo39.github.io/MiXLab/resources/mixlab.py \
                    -O /root/.ipython/mixlab.py"
    run(_spl(shellCmd))  # nosec
 
import importlib, mixlab
from google.colab import files  # pylint: disable=import-error #nosec
from mixlab import checkAvailable, runSh, rcloneConfigurationPath, prepareSession
 
 
def generateUploadList():
    prepareSession()
    if checkAvailable("/content/upload.txt"):
        runSh("rm -f upload.txt")
    runSh(
        f"rclone --config {rcloneConfigurationPath}/rclone.conf lsf {REMOTE}: --include '{QUERY_PATTERN}' --drive-shared-with-me --files-only --max-depth 1 > /content/upload.txt",
        shell=True,  # nosec
    )
 
 
def uploadLocalFiles():
    prepareSession()
    if MODE == "UTILS":
        filePath = "/root/.ipython/mixlab.py"
    elif MODE in ("RCONFIG", "RCONFIG_append"):
        filePath = f"{rcloneConfigurationPath}/rclone.conf"
    else:
        pass
 
    try:
      if checkAvailable(filePath):
        runSh(f"rm -f {filePath}")
      display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Upload rclone.conf from your local machine.</h2><br>"))
      uploadedFile = files.upload()
      fileNameDictKeys = uploadedFile.keys()
      fileNo = len(fileNameDictKeys)
      if fileNo > 1:
          for fn in fileNameDictKeys:
              runSh(f'rm -f "/content/{fn}"')
          return print("\nOnly upload one configuration file!")
      elif fileNo == 0:
          return print("\nFile upload cancelled.")
      elif fileNo == 1:
          for fn in fileNameDictKeys:
              if checkAvailable(f"/content/{fn}"):
                  if MODE == "RCONFIG_append":
                    import urllib
                    urllib.request.urlretrieve("https://shirooo39.github.io/MiXLab/resources/configurations/rclone/rclone.conf",
                                               "/usr/local/sessionSettings/rclone.conf")
                    with open(f"/content/{fn}", 'r+') as r:
                      new_data = r.read()
                      runSh(f'rm -f "/content/{fn}"')
                    with open(filePath, 'r+') as f:
                      old_data = f.read()
                      f.seek(0)
                      f.truncate(0)
                      f.write(old_data + new_data)
                    print("\nUpdate completed.")
                  else:
                    runSh(f'mv -f "/content/{fn}" {filePath}')
                    runSh(f"chmod 666 {filePath}")
                    runSh(f'rm -f "/content/{fn}"')
                    importlib.reload(mixlab)
                    !rm /content/upload.txt
                    clear_output()
                    print("rclone.conf has been uploaded to Colab!")
              return
      else:
          print("\nNo file is chosen!")
          return
    except:
      return print("\nFailed to upload!")
 
 
if MODE == "GENERATELIST":
    generateUploadList()
else:
    uploadLocalFiles()

### <font size=6>Mount/Unmount Rclone drive</font><br>

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Mount a remote drive as a local drive on a mountpoint </font>
# @markdown ---
Cache_Directory = "DISK" #@param ["RAM", "DISK"]
# ================================================================ #
 
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
 
if not os.path.exists("/root/.ipython/mixlab.py"):
  from shlex import split as _spl
  from subprocess import run
 
  shellCmd = "wget -qq https://shirooo39.github.io/MiXLab/resources/mixlab.py \
                  -O /root/.ipython/mixlab.py"
  run(_spl(shellCmd))
 
from mixlab import (
    runSh,
    prepareSession,
    rcloneConfigurationPath,
)
 
class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Available drive to mount/unmount:</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))
 
prepareSession()
content = open(f"{rcloneConfigurationPath}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)
 
if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'
 
def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {rcloneConfigurationPath}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --buffer-size 256M' \
      ' --transfers 10' \
      ' --vfs-cache-mode full' \
      ' --vfs-cache-max-age 0h0m1s' \
      ' --vfs-cache-poll-interval 0m1s' \
      f' --cache-dir {cache_path}' \
      ' --allow-other' \
      ' --daemon'
 
    if runSh(cmd, shell=True) == 0:
      print(f"The drive have been successfully mounted! - \t{mPoint}")
    else:
      print(f"Failed to mount the drive! - \t{mPoint}")
 
def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"The drive have been successfully unmounted! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)
 
ShowAC()

### <font size=6>Rclone Tools</font><br>

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] rclone</font>
Mode = "Copy" #@param ["Copy", "Move", "Sync", "Checker", "Deduplicate", "Remove Empty Directories", "Empty Trash"]
Source = "" #@param {type:"string"}
Destination = "" #@param {type:"string"}

#@markdown ---
#@markdown <center><h3><font color="#3399ff"><b>⚙️ Global Configuration ⚙️</b></font></h3></center>
Extra_Arguments = "--local-no-check-updated" #@param {type:"string"}
Compare = "Size & Mod-Time" #@param ["Size & Mod-Time", "Size & Checksum", "Only Mod-Time", "Only Size", "Only Checksum"]
Checkers = 8 #@param {type:"slider", min:1, max:40, step:1}
Transfers = 1 #@param {type:"slider", min:1, max:20, step:1}
Dry_Run = False #@param {type:"boolean"}
Do_not_cross_filesystem_boundaries = False
Do_not_update_modtime_if_files_are_identical = False #@param {type:"boolean"}
Google_Drive_optimization = True #@param {type:"boolean"}
Large_amount_of_files_optimization = True #@param {type:"boolean"}
Simple_Ouput = True #@param {type:"boolean"}
Skip_all_files_that_exist = False #@param {type:"boolean"}
Skip_files_that_are_newer_on_the_destination = False #@param {type:"boolean"}
Output_Log_File = "OFF" #@param ["OFF", "NOTICE", "INFO", "ERROR", "DEBUG"]

#@markdown <br><center><h3><font color="#3399ff"><b>↪️ Sync Configuration ↩️</b></font></h3></center>
Sync_Mode = "Delete during transfer" #@param ["Delete during transfer", "Delete before transfering", "Delete after transfering"]
Track_Renames = False #@param {type:"boolean"}

#@markdown <br><center><h3><font color="#3399ff"><b>💞 Deduplicate Configuration 💞</b></font></h3></center>
Deduplicate_Mode = "Interactive" #@param ["Interactive", "Skip", "First", "Newest", "Oldest", "Largest", "Rename"]
Deduplicate_Use_Trash = True #@param {type:"boolean"}

#@markdown ---
automatically_clear_cell_output = False  # @param{type: "boolean"}
# ================================================================ #

##### Importing the needed modules
import os
import IPython
from IPython.display import HTML, clear_output


##### Variable Declaration
# Optimized for Google Colaboratory
os.environ["bufferC"] = "--buffer-size 96M"

if Compare == "Size & Checksum":
    os.environ["compareC"] = "-c"
elif Compare == "Only Mod-Time":
    os.environ["compareC"] = "--ignore-size"
elif Compare == "Only Size":
    os.environ["compareC"] = "--size-only"
elif Compare == "Only Checksum":
    os.environ["compareC"] = "-c --ignore-size"
else:
    os.environ["compareC"] = ""

os.environ["sourceC"] = Source
os.environ["destinationC"] = Destination
os.environ["transfersC"] = "--transfers "+str(Transfers)
os.environ["checkersC"] = "--checkers "+str(Checkers)

if Skip_files_that_are_newer_on_the_destination == True:
    os.environ["skipnewC"] = "-u"
else:
    os.environ["skipnewC"] = ""
  
if Skip_all_files_that_exist == True:
    os.environ["skipexistC"] = "--ignore-existing"
else:
    os.environ["skipexistC"] = ""
  
if Do_not_cross_filesystem_boundaries == True:
    os.environ["nocrossfilesystemC"] = "--one-file-system"
else:
    os.environ["nocrossfilesystemC"] = ""
  
if Do_not_update_modtime_if_files_are_identical == True:
    os.environ["noupdatemodtimeC"] = "--no-update-modtime"
else:
    os.environ["noupdatemodtimeC"] = ""

if Large_amount_of_files_optimization == True:
    os.environ["filesoptimizeC"] = "--fast-list"
else:
    os.environ["filesoptimizeC"] = ""
  
if Google_Drive_optimization == True:
    os.environ["driveoptimizeC"] = "--drive-chunk-size 32M --drive-acknowledge-abuse --drive-keep-revision-forever"
else:
    os.environ["driveoptimizeC"] = ""
  
if Dry_Run == True:
    os.environ["dryrunC"] = "-n"
else:
    os.environ["dryrunC"] = ""
  
if Output_Log_File != "OFF":
    os.environ["statsC"] = "--log-file=/root/.rclone_log/rclone_log.txt"
else:
    if Simple_Ouput == True:
        os.environ["statsC"] = "-v --stats-one-line --stats=5s"
    else:
        os.environ["statsC"] = "-v --stats=5s"
  
if Output_Log_File == "INFO":
    os.environ["loglevelC"] = "--log-level INFO"
elif Output_Log_File == "ERROR":
    os.environ["loglevelC"] = "--log-level ERROR"
elif Output_Log_File == "DEBUG":
    os.environ["loglevelC"] = "--log-level DEBUG"
else:
    os.environ["loglevelC"] = ""

os.environ["extraC"] = Extra_Arguments

if Sync_Mode == "Delete during transfer":
    os.environ["syncmodeC"] = "--delete-during"
elif Sync_Mode == "Delete before transfering":
    os.environ["syncmodeC"] = "--delete-before"
elif Sync_Mode == "Delete after transfering":
    os.environ["syncmodeC"] = "--delete-after"
  
if Track_Renames == True:
    os.environ["trackrenamesC"] = "--track-renames"
else:
    os.environ["trackrenamesC"] = ""
  
if Deduplicate_Mode == "Interactive":
    os.environ["deduplicateC"] = "interactive"
elif Deduplicate_Mode == "Skip":
    os.environ["deduplicateC"] = "skip"
elif Deduplicate_Mode == "First":
    os.environ["deduplicateC"] = "first"
elif Deduplicate_Mode == "Newest":
    os.environ["deduplicateC"] = "newest"
elif Deduplicate_Mode == "Oldest":
    os.environ["deduplicateC"] = "oldest"
elif Deduplicate_Mode == "Largest":
    os.environ["deduplicateC"] = "largest"
elif Deduplicate_Mode == "Rename":
    os.environ["deduplicateC"] = "rename"
  
if Deduplicate_Use_Trash == True:
    os.environ["deduplicatetrashC"] = ""
else:
    os.environ["deduplicatetrashC"] = "--drive-use-trash=false"


##### rclone Execution
if Output_Log_File != "OFF" and Mode != "Config":
    !mkdir -p -m 666 /root/.rclone_log/
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Logging enabled, rclone will no longer display any output on the terminal.<br>Please wait until the cell stop by itself.</h2></center><br>"))

if Mode == "Copy":
    !rclone --config=/root/.config/rclone/rclone.conf copy "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Move":
    !rclone --config=/root/.config/rclone/rclone.conf move "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Sync":
    !rclone --config=/root/.config/rclone/rclone.conf sync "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $syncmodeC $trackrenamesC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Checker":
    !rclone --config=/root/.config/rclone/rclone.conf check "$sourceC" "$destinationC" $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Deduplicate":
    !rclone --config=/root/.config/rclone/rclone.conf dedupe "$sourceC" $checkersC $statsC $loglevelC --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Remove Empty Directories":
    !rclone --config=/root/.config/rclone/rclone.conf rmdirs "$sourceC" $statsC $loglevelC $dryrunC $extraC
elif Mode == "Empty Trash":
    !rclone --config=/root/.config/rclone/rclone.conf cleanup "$sourceC" $statsC $loglevelC $dryrunC $extraC


##### Log Output
if Output_Log_File != "OFF" and Mode != "Config":

    ##### Rename log file and output settings.
    !mv /root/.rclone_log/rclone_log.txt /root/.rclone_log/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt
    with open("/root/.rclone_log/" + Mode + "_settings.txt", "w") as f:
        f.write("Mode: " + Mode + \
            "\nCompare: " + Compare + \
            "\nSource: \"" + Source + \
            "\"\nDestination: \"" + Destination + \
            "\"\nTransfers: " + str(Transfers) + \
            "\nCheckers: " + str(Checkers) + \
            "\nSkip files that are newer on the destination: " + str(Skip_files_that_are_newer_on_the_destination) + \
            "\nSkip all files that exist: " + str(Skip_all_files_that_exist) + \
            "\nDo not cross filesystem boundaries: " + str(Do_not_cross_filesystem_boundaries) + \
            "\nDo not update modtime if files are identical: " + str(Do_not_update_modtime_if_files_are_identical) + \
            "\nDry-Run: " + str(Dry_Run) + \
            "\nOutput Log Level: " + Output_Log_File + \
            "\nExtra Arguments: \"" + Extra_Arguments + \
            "\"\nSync Moden: " + Sync_Mode + \
            "\nTrack Renames: " + str(Track_Renames) + \
            "\nDeduplicate Mode: " + Deduplicate_Mode + \
            "\nDeduplicate Use Trash: " + str(Deduplicate_Use_Trash))

    ##### Compressing log file.
    !rm -f /root/rclone_log.zip
    !zip -r -q -j -9 /root/rclone_log.zip /root/.rclone_log/
    !rm -rf /root/.rclone_log/
    !mkdir -p -m 666 /root/.rclone_log/

    ##### Send Log
    if os.path.isfile("/root/rclone_log.zip") == True:
        try:
            files.download("/root/rclone_log.zip")
            !rm -f /root/rclone_log.zip
            display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Sending log to your browser...</h2><br></center>"))
        except:
            !mv /root/rclone_log.zip /content/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).zip
            display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">You can use file explorer to download the log file.</h2><br><img src=\"https://minormole.github.io/RcloneLab/res/rclonelab/01.png\"><br></center>"))
    else:
        clear_output()
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">There is no log file.</h2><br></center>"))
    

### Operation has been successfully completed.
if Mode != "Config":
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">✅ Operation has been successfully completed.</h2><br></center>"))


##### Automatically clear terminal output if the checkbox's value on the top is set to True.
if automatically_clear_cell_output is True:
    clear_output()
else:
	pass##### Automatically clear terminal output if the checkbox's value on the top is set to True.
if automatically_clear_cell_output is True:
    clear_output()
else:
	pass

# **<font color='blue'>QBITTORRENT</font>**
# <img src='https://i.ibb.co/jR1vq3s/pcgamescollections2021-q-Bittorrent.png' height="45" alt="qBittorrent"/>

* Install localtunnel **first** and then install qbittorrent, then it'll give you a URL for the WEB-UI.

In [ ]:
#@markdown <h3>←INSTALL LOCALTUNNEL</h3>
!npm install -g localtunnel
!npm install -g npm

In [ ]:
#@markdown <h3>←INSTALL QBITTORRENT</h3>

import os, time, urllib.request, json; from IPython.display import clear_output


if os.path.isfile("/usr/bin/qbittorrent-nox") == False:
    get_ipython().system_raw("apt update -qq -y && yes "" | add-apt-repository ppa:qbittorrent-team/qbittorrent-stable")
    get_ipython().system_raw("apt install qbittorrent-nox")
    get_ipython().system_raw("mkdir -p -m 666 /{content/qBittorrent,root/{.qBittorrent_temp,.config/qBittorrent}} && curl -s https://pastebin.com/raw/CM3HcKwG -o /root/.config/qBittorrent/qBittorrent.conf")
    print("qBittorrent successfully installed.")
    clear_output(wait=True)
else:
    print("qBittorrent already installed. Skipping...")
    clear_output(wait=True)
    !pkill qbittorrent-nox
    
!qbittorrent-nox -d --webui-port=5454
print("qBittorrent started ")
clear_output(wait=True)

if os.path.isfile("/usr/bin/rclone") == False:
    get_ipython().system_raw('curl https://rclone.org/install.sh | sudo bash &')
    print("Rclone successfully installed")
    clear_output(wait=True)
else:
    print("Rclone already installed. Skipping...")
    clear_output(wait=True)

!lt --port 5454

# **<font color='blue'>__GDRIVE TO MEGA__</font>**
# <img src='https://i.ibb.co/drrqpm2/pcgamescollections2021-google-drive-to-mega.png' height="60" alt="MEGA-logo"/>

In [ ]:
# @markdown <h3>Input your Mega Id and Password</h3><br>


from functools import wraps
import errno
import os
import signal
import subprocess
import shlex


class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)


# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# INPUT YOUR MEGA ID

USERNAME = "your email.com"  # @param {type:"string"}
PASSWORD = "your password"  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega IDs.")


In [ ]:

# @markdown <h3>←TRANSFER NOW</h3><br>
# TRANSFER NOW

import time
import subprocess
import contextlib
from IPython.display import clear_output

PATH_TO_FILE = "/content/drive/"  # @param {type:"string"}
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']


def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    cmd = ['mega-put', PATH_TO_FILE, '/colab']
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

try:
    transfare()
except FileNotFoundError:
    print("Login your account!")


In [ ]:
# @markdown <h3>←LOGOUT</h3><br>

!mega-logout

# **<font color='blue'>__ARCHIVE.ORG UPLOADER__</font>**

In [ ]:
#@markdown <h3>←INSTALL INTERNETARCHIVE</h3>
!pip install internetarchive
!curl -LO https://archive.org/download/ia-pex/ia
!chmod +x ia

In [ ]:
#@markdown <h3>←INPUT CREDENTIALS</h3>
!ia configure

In [ ]:
#@markdown <h3>←START UPLOAD</h3>
File_path = "" #@param {type:"string"}
Identifier = "" #@param {type:"string"}
media_type = "movies" #@param ["movies", "documents"]
!ia upload {Identifier} {File_path} --metadata="mediatype:{media_type}" --retries 10

# **<font color='blue'>__MEDIAFIRE UPLOADER__</font>**

In [ ]:
#@markdown <h3>👈 Run this cell to install requirements</h3>
!echo -n "Installing mediafire-ul..." && mkdir -p sample_data/mediafire-ul &> /dev/null && git clone https://github.com/JayBrown/mfcmd.py---MediaFire-uploader.git sample_data/mediafire-ul &> /dev/null && pip install pymediafire &> /dev/null && pip install mediafire &> /dev/null && echo " Done!"


In [ ]:
#@markdown <h3>⬆️ MediaFire Uploader</h3>
#@markdown The MediaFire API is pretty buggy, and you might receive an error, because the server hasn't finished calculating the file's checksum yet; in that case, wait a couple of seconds (depending on the filesize) and repeat the command: it will not upload, but instead detect the already uploaded file and return the download URL.
Email_Address = "" #@param {type:"string"}
Password = "" #@param {type:"string"}
File_path = "" #@param {type:"string"}
mfemail, mfpassword = Email_Address, Password

def get_file_info(filepath):
  splited = filepath.split("/")
  filename=splited[-1]
  pathname="/".join(splited[0:-1])
  return filename, pathname

filename, pathname = get_file_info(File_path)
print("Uploading", filename+"...")
!cd /content/sample_data/mediafire-ul && python3 mfcmd.py -e $mfemail -p $mfpassword -f $File_path 2>/dev/null
print("Done!")

# **<font color='blue'>__MEDIA INFO__</font>**

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=5>← Install FFmpeg</font>

from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/cheems/FFmpeg-for-GDrive/master/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt install mediainfo')
os.system('apt-get install ffmpeg')
clear_output()
print('Installation finished.')

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=5>← Media Info</font>

path = "" #@param {type:"string"}
save_txt = True #@param {type:"boolean"}
import os, uuid, re, IPython
import ipywidgets as widgets
import time

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output, drive

def mediainfo():
  display(HTML("<br>"))
#   print(path.split("/")[::-1][0])
  display(HTML("<br>"))
#   media = !mediainfo "$path"
#   media = "\n".join(media).replace(os.path.dirname(path)+"/", "")
  get_ipython().system_raw("""mediainfo --LogFile="/root/.nfo" "$path" """)
  with open('/root/.nfo', 'r') as file:
    media = file.read()
    media = media.replace(os.path.dirname(path)+"/", "")
  print(media)
  get_ipython().system_raw("rm -f '/root/.nfo'")
  
  if save_txt:
    txt = path.rpartition('.')[0] + ".txt"
    if os.path.exists(txt):
      get_ipython().system_raw("rm -f '$txt'")

    with open(txt, 'a+') as file:
      file.write("\n\n")
      file.write(media)

while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output()
    
if not os.path.exists("/usr/bin/mediainfo"):
  get_ipython().system_raw("apt-get install mediainfo")
  
mediainfo()

# <img src="https://img.icons8.com/dotty/80/000000/archive.png" height="45" align="left" alt=""/><img src="" height="45" align="left" alt=""/><b><font size=6>Create / Extract Archive</font></b>

##__Rar Files/Folders__

In [ ]:
# @markdown <h3>←INSTALL RAR</h3><br>
!apt-get install rar

In [ ]:
# @markdown <h3>←CREATE ARCHIVE</h3><br>
Input_File_or_Folder_Path = ""  # @param {type:"string"}
Output_File_Name = ""  # @param {type:"string"}
Split_Volume = "0"  # @param {type:"string"}

!rar a -v{Split_Volume}m "{Input_File_or_Folder_Path}/{Output_File_Name}" "{Input_File_or_Folder_Path}"

#@markdown #####_*Split_Volume Sample:_
#@markdown #####_100 = 100 MegaBytes._